In [1]:
import numpy as np

import includes.bdcs as bdcs
import includes.fde as fde

import isde
import isde.estimators
# import includes.isde as isde

from sklearn.covariance import empirical_covariance
import pandas as pd

np.random.seed(0)

In [2]:
import pandas as pd

def show_tables(dict_output, factor, prec, score):
    ''' Display tables : score_means, score_std and running times
    
    dict_output (dictionnary): dictionnary containing scores and running time for different methods and data
    factor (real): multiply scores by a constant factor
    prec (integer) : how many decimals are kept
    score (str) : dict_output key corresponding to score
    '''
    exps = [i for i  in dict_output]
    Ss = [i for i  in dict_output[exps[0]]]

    def _mean(exp, struct, what):
        return np.mean(dict_output[exp][struct][what])

    def _std(exp, struct, what, display=False):
        return np.std(dict_output[exp][struct][what])

    def _list(exp, struct, what, display=False):
        return dict_output[exp][struct][what]

    import pandas as pd

    df_means = pd.DataFrame(0.0, index=exps, columns=Ss)
    df_stds = pd.DataFrame(0.0, index=exps, columns=Ss)
    df_running_times = pd.DataFrame(0.0, index=exps, columns=Ss)

    for exp in df_means.index:
        for struct in df_means.columns:
            df_means[struct][exp] = round(_mean(exp=exp, struct=struct, what=score) * factor, prec)
            df_stds[struct][exp] = round(_std(exp=exp, struct=struct, what=score) * np.abs(factor), prec)
            df_running_times[struct][exp] = round(_mean(exp=exp, struct=struct, what='exec_time'), 3)


    print("Means :")
    print(df_means)
    print("\n------------------------------------------------------\n")
    print("Standard deviations")
    print(df_stds)
    print("\n------------------------------------------------------\n")
    print("Mean running times")
    print(df_running_times)
    print("\n------------------------------------------------------\n")
    print("Recovery : ")
    for struct in Ss:
        for exp in exps:
            if 'partition' in dict_output[exp][struct] or 'graph' in dict_output[exp][struct]:
                if 'partition' in dict_output[exp][struct]:
                    key = 'partition'
                else:
                    key = 'graph'
                print("{} / {}".format(exp, struct))
                l = _list(exp, struct, key)
                for i in l:
                    print(i)
                    
                print("\n------------------------------------------------------\n")
                

## Gaussian setup

In [3]:
# Function to simulate data
def Sigma(struct, sigma_value):
    #Matrix abd structure as numpy array with values for alpha and sigma
    
    d = np.sum(struct)
    M = np.zeros(shape=(d, d))
    
    a = 0
    for i, s in enumerate(np.cumsum(struct)):
        b = s
        M[a:b, a:b] = sigma_value * np.ones(shape=(b - a, b - a))
        a = b
    
    np.fill_diagonal(M, 1)
    return M


# Compute KL loss
def KL(Sigma1, Sigma2):
    
    Prec1 = np.linalg.inv(Sigma1)
    Prec2 = np.linalg.inv(Sigma2)
    
    B = np.dot(Prec2 - Prec1, Sigma1)

    v = np.linalg.eig(B)[0]
    return np.sum(v - np.log(1 + v)) / 2

In [4]:
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

# import isde
from time import time

import pickle
import os

Ss = [[2, 2], [4, 4, 1], [4, 3, 2, 3], [4, 4, 3, 3, 2]]
N = 6000
sigma = 0.7
m = int(N/2)
n = int(N/2)

reps = 5

output_ggm = {i : {} for i in ['ISDE', 'BDCS', 'Empirical Covariance']}

for struct in Ss:
    
    print(struct)
    d = np.sum(struct)
    
    output_ggm['ISDE'][str(struct)] = {'KL' : [], 'exec_time' : [], 'partition' : []}
    output_ggm['BDCS'][str(struct)] = {'KL' : [], 'exec_time' : [], 'partition' : []}
    output_ggm['Empirical Covariance'][str(struct)] = {'KL' : [], 'exec_time' : []}
    
    for j in range(reps):
        print('{}/{}'.format(j+1, reps))
        
        cov = Sigma(struct=struct, sigma_value=sigma)
        X = np.random.multivariate_normal(mean=np.zeros(d), cov=cov, size=N)
      
        #Empirical Covariance
        start = time()
        cov_empcov = empirical_covariance(X)
        end = time()
        output_ggm['Empirical Covariance'][str(struct)]['KL'].append(KL(cov, cov_empcov))
        output_ggm['Empirical Covariance'][str(struct)]['exec_time'].append(end-start)
        
        #BDCS
        start = time()
        A = bdcs.BDCS()
        cov_bdcs, partition_bdcs = A.fit(X)
        end=time()
        output_ggm['BDCS'][str(struct)]['KL'].append(KL(cov, cov_bdcs))
        output_ggm['BDCS'][str(struct)]['exec_time'].append(end-start)
        output_ggm['BDCS'][str(struct)]['partition'].append(str(partition_bdcs))

        #ISDE
        start = time()
        _, partition_isde, param = isde.ISDE(X=X, m=m, n=n, k=d,
                                             multidimensional_estimator=isde.estimators.EmpCovariance,
                                             do_optimization=True, verbose=False)
        

        end = time()
        cov_isde = bdcs.empirical_covariance_partition(X, partition_isde)            
        output_ggm['ISDE'][str(struct)]['KL'].append(KL(cov, cov_isde))
        output_ggm['ISDE'][str(struct)]['exec_time'].append(end-start)
        output_ggm['ISDE'][str(struct)]['partition'].append(str(partition_isde))
        

[2, 2]
1/5
2/5
3/5
4/5
5/5
[4, 4, 1]
1/5
2/5
3/5
4/5
5/5
[4, 3, 2, 3]
1/5
2/5
3/5
4/5
5/5
[4, 4, 3, 3, 2]
1/5
2/5
3/5
4/5
5/5


In [5]:
show_tables(output_ggm, factor=1e3, prec=5, score='KL')

Means :
                       [2, 2]  [4, 4, 1]  [4, 3, 2, 3]  [4, 4, 3, 3, 2]
ISDE                  0.60136    1.87957       2.85246          5.29782
BDCS                  0.60136    1.72244       2.62826          4.42350
Empirical Covariance  0.80475    3.61933       6.87859         12.63409

------------------------------------------------------

Standard deviations
                       [2, 2]  [4, 4, 1]  [4, 3, 2, 3]  [4, 4, 3, 3, 2]
ISDE                  0.20930    0.52118       0.59721          0.95969
BDCS                  0.20930    0.46439       1.00837          1.80356
Empirical Covariance  0.19542    0.53006       0.84129          0.83436

------------------------------------------------------

Mean running times
                      [2, 2]  [4, 4, 1]  [4, 3, 2, 3]  [4, 4, 3, 3, 2]
ISDE                   0.020      0.346         3.005           60.099
BDCS                   0.036      0.113         0.251            0.612
Empirical Covariance   0.000      0.000         0.

## NonParametric Setup

In [6]:
def logdensity_from_partition(X, X_eval, partition, parameters, estimator):
    M = len(X_eval)
    log_density = np.zeros(len(X_eval))
    for i, S in enumerate(partition):

        loc_param = parameters[i]
        f = estimator(**loc_param)
        log_density += f.score_samples(grid_points=X[:, S], eval_points=X_eval[:, S])

    return log_density

In [7]:
def nonparametric_data(N, struct):
    
    d = np.sum(struct)
    X = np.zeros(shape=(N, d))
    cs = np.cumsum(struct)
    for i, s in enumerate(struct):
        
        if s == 1:
            X[:, cs[i]-1] = np.random.rand(N)
        if s == 2:
            X[:, cs[i]-2:cs[i]] = datasets.make_circles(n_samples=N, factor=.5, noise=.05)[0]
            
        if s == 3:
            X1 = np.random.binomial(n=1, p=0.5, size=N)
            X2 = np.random.binomial(n=1, p=0.5, size=N)
            X3 = np.abs(X2 - X1)
            
            X[:, cs[i]-3:cs[i]] = np.stack([X1, X2, X3], axis=1) + np.random.multivariate_normal(mean=np.zeros(3), cov=0.08 * np.identity(3), size=N)
            
            
        if s >= 4:
            Y = np.random.binomial(n=1, p=0.5, size=N)
            X[np.where(Y==0), cs[i]-s:cs[i]] = np.random.multivariate_normal(np.zeros(s), 0.2*np.identity(s), size=np.sum(Y==0))
            X[np.where(Y==1), cs[i]-s:cs[i]] = np.random.multivariate_normal(np.ones(s), 0.2*np.identity(s), size=np.sum(Y==1))
        
    return MinMaxScaler().fit_transform(X)

In [8]:
Ss = [[2,2,1], [3,3,3], [4, 4, 2, 2]]
N = 5000
M = 5000
hs = np.logspace(-2, 1, 30)
m = int(N/2)
n = int(N/2)

reps = 5

output_nonparam = {i : {} for i in ['CVKDE', 'FDE', 'ISDE_CVKDE', 'ISDE_fixed_h']}

for struct in Ss:
    
    d = np.sum(struct)
    print(struct)
    
    output_nonparam['CVKDE'][str(struct)] = {'log_likelihood' : [], 'exec_time' : []}
    output_nonparam['FDE'][str(struct)] = {'log_likelihood' : [], 'exec_time' : [], 'graph' : []}
    output_nonparam['ISDE_CVKDE'][str(struct)] = {'log_likelihood' : [], 'exec_time' : [], 'partition' : []}
    output_nonparam['ISDE_fixed_h'][str(struct)] = {'log_likelihood' : [], 'exec_time' : [], 'partition' : []}
    
    for j in range(reps):
        print('{}/{}'.format(j+1, reps))
        
        X = nonparametric_data(N=N, struct=struct)
        X_validation = nonparametric_data(N=M, struct=struct)
        
        #CVKDE
        start = time()
        est, _ = isde.estimators.CVKDE(X, params={"hs": hs, "n_fold" : 5})
        end = time()
        ll_cvkde = np.mean(est.score_samples(grid_points=X, eval_points=X_validation))
        output_nonparam['CVKDE'][str(struct)]['log_likelihood'].append(ll_cvkde)
        output_nonparam['CVKDE'][str(struct)]['exec_time'].append(end-start)
        
        #ISDE CVKDE
        start = time()
        by_subsets, part, param = isde.ISDE(X=X, m=m, n=n, k=d,
                                   multidimensional_estimator=isde.estimators.CVKDE, hs=hs, n_fold=5,
                                   do_optimization=True, verbose=False)
        
        end = time()
        ll_isde = np.mean(logdensity_from_partition(X=X, X_eval=X_validation, partition=part, parameters=param,
                                                    estimator=isde.estimators.GaussianKDE))
        output_nonparam['ISDE_CVKDE'][str(struct)]['log_likelihood'].append(ll_isde)
        output_nonparam['ISDE_CVKDE'][str(struct)]['exec_time'].append(end-start)
        output_nonparam['ISDE_CVKDE'][str(struct)]['partition'].append(str(part))
        
        #FDE
        start = time()
        graph = fde.FDE(X, bw_bysubsets=True, by_subsets=by_subsets)
        end=time()
        ll_fde = np.mean(fde.logdensity_from_tree(X, X_validation, graph, bw_bysubsets=True, by_subsets=by_subsets))
#         ll_fde = np.mean(logdensity_from_tree(X=X, X_eval=X_validation, graph=graph, h1=.05, h2=.05))
        output_nonparam['FDE'][str(struct)]['log_likelihood'].append(ll_fde)
        output_nonparam['FDE'][str(struct)]['exec_time'].append(end-start)
        output_nonparam['FDE'][str(struct)]['graph'].append(str(graph))

        
        
        #ISDE Fixed h
        start = time()
        _, part, param = isde.ISDE(X=X, m=m, n=n, k=d,
                                   multidimensional_estimator=isde.estimators.KDE_fixed_h, h=.05,
                                   do_optimization=True, verbose=False)
        end = time()
        ll_isde_fixedh = np.mean(logdensity_from_partition(X=X, X_eval=X_validation, partition=part,
                                                           parameters=param, estimator=isde.estimators.GaussianKDE))
        output_nonparam['ISDE_fixed_h'][str(struct)]['log_likelihood'].append(ll_isde_fixedh)
        output_nonparam['ISDE_fixed_h'][str(struct)]['exec_time'].append(end-start)
        output_nonparam['ISDE_fixed_h'][str(struct)]['partition'].append(str(part))

[2, 2, 1]
1/5
2/5
3/5
4/5
5/5
[3, 3, 3]
1/5
2/5
3/5
4/5
5/5
[4, 4, 2, 2]
1/5
2/5
3/5
4/5
5/5


In [9]:
show_tables(dict_output=output_nonparam, factor=1, prec=5, score='log_likelihood')

Means :
              [2, 2, 1]  [3, 3, 3]  [4, 4, 2, 2]
CVKDE           0.56494    3.48505       3.96350
FDE             1.82914    2.87710       5.89451
ISDE_CVKDE      1.82914    4.04658       6.29758
ISDE_fixed_h    1.00747    4.03716       5.54726

------------------------------------------------------

Standard deviations
              [2, 2, 1]  [3, 3, 3]  [4, 4, 2, 2]
CVKDE           0.02773    0.10593       0.16274
FDE             0.08247    0.13758       0.33114
ISDE_CVKDE      0.08247    0.14670       0.25155
ISDE_fixed_h    0.02271    0.13511       0.25142

------------------------------------------------------

Mean running times
              [2, 2, 1]  [3, 3, 3]  [4, 4, 2, 2]
CVKDE             1.335      1.555         1.767
FDE               0.001      0.001         0.001
ISDE_CVKDE       15.961    297.235      2611.502
ISDE_fixed_h      0.143      2.392        21.217

------------------------------------------------------

Recovery : 
FDE / [2, 2, 1]
[[2, 3], [0, 1]]
[[

In [10]:
import pickle

pickle.dump(output_ggm, open("outputs/output_ggm", "wb"))
pickle.dump(output_nonparam, open("outputs/output_nonparam", "wb")) 